In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
# Import pyMC3 and also arviz for visualisation
import pymc as pm
import arviz as az
import sympy as sp
import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
plt.style.use('bmh')
import symseaman as ss
from symseaman.substitute_dynamic_symbols import lambdify, remove_bis_eq, run, standard_substitutes
from symseaman.seaman_symbols import *
from vct.bis_system import BisSystem

In [ ]:
import sympy as sp
from symseaman.seaman_symbols import *
from symseaman.substitute_dynamic_symbols import run, lambdify, standard_substitutes, remove_bis, remove_bis_eq, standard_substitutes_eq, lower_and_abs
from symseaman.seaman_symbol import BisSymbol
import symseaman as ss
from symseaman.seaman_symbol import expand_bis
from symseaman.shipdict import ShipDict
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', -1)
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 12, 4 
plt.style.use('bmh')
import vct.bokeh_plotting as bokeh_plotting
from numpy.testing import assert_almost_equal
from scipy.interpolate import interp1d
from symseaman.calculate import calculate_ship
from symseaman.to_matrix import matrix_form, matrix_form_standard, exclude, exclude_derivatives
from functools import reduce
from operator import add
from vct.regression_ols import Regression, RegressionPipeline
import dill

In [ ]:
# Read configs:
import anyconfig
globals_path = "../conf/base/globals.yml"
global_variables = anyconfig.load(globals_path)
ships = global_variables["ships"]
from vct.bis_system import BisSystem
import statsmodels.api as sm
from scipy.stats import norm

In [ ]:
%reload_kedro
ship_name=ships[0]
shipdict = catalog.load(f"{ship_name}.shipdict_base")
ship_dict_from_regression = catalog.load(f"{ ship_name }.ship_dict_from_regression")

df_VCT = catalog.load(f"{ship_name}.df_VCT")
df_VCT['g'] = df_VCT['g'].fillna(method='pad')
df_VCT['rho'] = df_VCT['rho'].fillna(method='pad')

thrust_keys = catalog.load(f"params:{ship_name}.thrust_keys")

if len(thrust_keys) > 0:
    df_VCT['tprop'] = df_VCT[thrust_keys[0]]
else:
    df_VCT['tprop'] = 0
    thrust_keys=['tprop']
    
derivatives = catalog.load(f"{ship_name}.derivatives")["OLS"]
derivatives_std = catalog.load(f"{ship_name}.derivatives_std")["OLS"]

In [ ]:
lpp_ = shipdict.main_data['l']
t_ = shipdict.design_particulars['ta']
disp_ = shipdict.design_particulars['disp']
units={'fy_rudders':'force',
       'fy_rudder':'force',
       'tprop':'force',
      }
bis_system = BisSystem(lpp=lpp_, volume=disp_, units=units)

interesting = list((set(bis_system.qd.keys()) & set(df_VCT.columns)) |  (set(df_VCT.columns) & set(['test type', 'model_name','fy_rudders','fy_rudder','tprop'])))
interesting = list(set(interesting))
df_bis = bis_system.df_to_bis(df_VCT[interesting])

### $Y_{T\delta}$

In [ ]:
class Regression_ytd(Regression):
    _eq = ss.equations.sway.eq_expanded.subs([
    (phi.bis,0),
    (r_w.bis,0),
    (v_w.bis,0),
    #(Y_uudelta, 0),
    ])
    intercept=False

In [ ]:
pre_set_derivatives_0 = {
    's':shipdict.rudder_coeff_data['s'],
    'kv':shipdict.rudder_coeff_data['kv'],
    'kr':shipdict.rudder_coeff_data['kr'],
    'ktyh':derivatives['ktyh'],
    'xxprop':derivatives['xxprop'],
}
pipeline = RegressionPipeline(shipdict=shipdict, pre_set_derivatives=pre_set_derivatives_0)

mask = df_bis['test type'].isin(['Thrust variation','Rudder angle'])
df_ = df_bis.loc[mask]

pipeline['ytd'] = Regression_ytd(df=df_)

In [ ]:
regression = pipeline['ytd']

In [ ]:
y,X=regression.label_and_features(derivatives=pre_set_derivatives_0, meta_data=pipeline.meta_data)

In [ ]:
X

In [ ]:
regression.fit(derivatives=pre_set_derivatives_0, meta_data=pipeline.meta_data)

In [ ]:
data = df_
basic_model = pm.Model()

columns = list(X.columns)

sigmas_ = [derivatives_std[key] for key in columns]
mus_ = [derivatives[key] for key in columns]
sigma_ = 0.001*data['fy'].abs().max()

with basic_model:
    # Priors for unknown model parameters
    parameters = pm.Normal("parameters", mu=mus_, 
                           sigma=sigmas_, 
                           shape=len(columns))
    
    sigma = pm.HalfNormal("sigma", sigma=sigma_)

    # Expected value of outcome
    mu = pm.math.dot(X, parameters)

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("fy_obs", mu=mu, sigma=sigma, observed=y)
    #ν = pm.Uniform("ν", 1, 10)
    #Y_obs = pm.StudentT("fy_obs", nu=ν, mu=mu, sigma=sigma, observed=y)

In [ ]:
with basic_model:
    # draw 1000 posterior samples
    trace = pm.sample(draws=1000)

In [ ]:
with basic_model:
    az.plot_posterior(trace,
                  var_names=['parameters','sigma'],
                  textsize=18,
                  point_estimate='mean',
                  rope_color='black')

In [ ]:
az.summary(trace)

In [ ]:
means = (trace.posterior['parameters'].data[0].mean(axis=0) + trace.posterior['parameters'].data[1].mean(axis=0))/2
means = {key:mean for key,mean in zip(columns,means)}

In [ ]:
means

In [ ]:
stds = (trace.posterior['parameters'].data[0].std(axis=0) + trace.posterior['parameters'].data[1].std(axis=0))/2
stds = {key:std for key,std in zip(columns,stds)}

In [ ]:
stds

In [ ]:
regression.regression.bse['ytd']

In [ ]:
regression.regression.bse['yuud']

In [ ]:
derivatives_std['ytd']

In [ ]:
derivatives_std['yuud']

In [ ]:
df_parameters = pd.concat([
pd.Series(means, name='Bayesian'),
pd.Series({key:derivatives[key] for key in means.keys()}, name='splitted OLS'),
pd.Series({key:regression.regression.params[key] for key in means.keys()}, name='OLS'),
    
], axis=1).transpose()

In [ ]:
df_parameters